In [3]:
import pandas as pd

# 读取 JSON 文件
df_meta = pd.read_json("../query_bookreview/ground_truth_dataset/meta_gt.json", lines=True)
df_review = pd.read_json("../query_bookreview/ground_truth_dataset/review_gt.json", lines=True)

In [2]:
df_meta

,title,subtitle,author,rating_number,features,description,price,store,categories,details,parent_asin
0,Chaucer,"Hardcover – Import, January 1, 2004",{'avatar': 'https://m.media-amazon.com/images/...,29,[],[],8.23,Peter Ackroyd (Author),"[Books, Literature & Fiction, History & Critic...",{'Publisher': 'Chatto & Windus; First Edition ...,0701169850
1,Notes from a Kidwatcher,First Edition,{'avatar': 'https://m.media-amazon.com/images/...,1,[Contains 23 selected articles by this influen...,"[About the Author, SANDRA WILDE, Ph.D., is wid...",3.52,Sandra Wilde (Editor),"[Books, Reference, Words, Language & Grammar]",{'Publisher': 'Heinemann; First Edition (May 2...,0435088688
2,Service: A Navy SEAL at War,"Hardcover – May 8, 2012",{'avatar': 'https://m.media-amazon.com/images/...,3421,"[Marcus Luttrell, author of the #1 bestseller,...","[Review, Praise for SERVICE""An action-packed.....",17.17,"Marcus Luttrell (Author), James D. Hornfischer","[Books, Biographies & Memoirs, Leaders & Notab...","{'Publisher': 'Little, Brown and Company; 1st ...",0316185361
3,Monstrous Stories #4: The Day the Mice Stood S...,"Paperback – October 29, 2013",None,40,"[Funny, light-hearted monster stories that are...",[],7.43,Dr. Roach (Author),"[Books, Children's Books, Science Fiction & Fa...",{'Publisher': 'Scholastic Paperbacks; Reprint ...,0545425573
4,Parker & Knight,Kindle Edition,{'avatar': 'https://m.media-amazon.com/images/...,381,"[From REMINGTON KANE, the author of The Taken!...",[],0.00,Remington Kane (Author) Format: Kindle Edition,"[Books, Mystery, Thriller & Suspense, Thriller...","{'Publication date': 'May 18, 2014', 'Language...",B00KFOP3RG
...,...,...,...,...,...,...,...,...,...,...,...
195,The Secret Thoughts of an Unlikely Convert : A...,"Paperback – January 1, 2012",{'avatar': 'https://m.media-amazon.com/images/...,2645,"[Rosaria, by the standards of many, was living...",[],14.00,Rosaria Champagne Butterfield (Author),"[Books, Biographies & Memoirs, Leaders & Notab...",{'Publisher': 'Crown & Covenant; 1st edition (...,1884527388
196,THE ESSENTIAL GUIDE TO WRITING WELL AND GETTIN...,"Paperback – August 9, 2017",{'avatar': 'https://m.media-amazon.com/images/...,3,"[Over the course of many years, college studen...","[About the Author, Robert Banfelder is an awar...",29.99,Robert Banfelder (Author),"[Books, Education & Teaching]",{'Publisher': 'Broadwater Books; 1st edition (...,0991591291
197,SCUBA SHARK - Diving - Vinyl Car Decal Sticker...,None,None,16,[],[],NaN,None,"[Books, Sports & Outdoors, Water Sports]","{'Item Weight': '3.04 ounces', 'Dimensions': '...",B001INQIPE
198,la_guerra_secreta,"Paperback – January 1, 2004",{'avatar': 'https://m.media-amazon.com/images/...,6,"[1963,El complot,da culminacion a la trilogia,...",[],NaN,Spanish Edition fabia...,"[Books, Libros en español]",{'Publisher': 'editorial de ciencia sociales.;...,9590606148


In [3]:
import pandas as pd
import ast

# 如果 details 是字符串，需要先转为 dict
def parse_details(details):
    if isinstance(details, dict):
        return details
    try:
        return ast.literal_eval(details)
    except:
        return {}

df_meta["details_dict"] = df_meta["details"].apply(parse_details)

# 提取 Language 字段
df_meta["language"] = df_meta["details_dict"].apply(lambda d: d.get("Language") if isinstance(d, dict) else None)

# 丢弃 Language 为 None 的
df_lang = df_meta.dropna(subset=["language"]).copy()

# 可选：查看语言统计数量（频次）
language_counts = df_lang["language"].value_counts()

print(language_counts)


language
English               183
Spanish                 3
English, Arabic         1
English, Ukrainian      1
Name: count, dtype: int64


In [5]:
import pandas as pd
import ast
import re

# 安全解析 details 字段
def parse_details(details):
    if isinstance(details, dict):
        return details
    try:
        return ast.literal_eval(details)
    except:
        return {}

df_meta["details_dict"] = df_meta["details"].apply(parse_details)

# 提取 Publisher 字符串
df_meta["publisher_str"] = df_meta["details_dict"].apply(
    lambda d: d.get("Publisher") if isinstance(d, dict) else None
)

# 提取 publisher 字符串中所有 4 位年份，并取最早一个（如存在）
def extract_year(publisher):
    if not isinstance(publisher, str):
        return None
    years = re.findall(r'(\d{4})', publisher)
    years = [int(y) for y in years if 1500 <= int(y) <= 2025]
    return min(years) if years else None

df_meta["publish_year"] = df_meta["publisher_str"].apply(extract_year)

# 丢弃没有年份的记录
df_year = df_meta.dropna(subset=["publish_year"]).copy()

# 按 decade 聚类
df_year["decade"] = (df_year["publish_year"] // 10) * 10

# 统计 top 5 decades
top5_decades = df_year["decade"].value_counts().sort_values(ascending=False)

# 打印结果
print("📚 出版年代 Top 5：")
print(top5_decades)


📚 出版年代 Top 5：
decade
2010.0    84
2000.0    46
2020.0    21
1990.0    17
1980.0    11
1930.0     2
1970.0     2
1940.0     1
1880.0     1
Name: count, dtype: int64


In [7]:
import pandas as pd
import ast
import re

# 安全解析 details 字段
def parse_details(details):
    if isinstance(details, dict):
        return details
    try:
        return ast.literal_eval(details)
    except:
        return {}

df_meta["details_dict"] = df_meta["details"].apply(parse_details)

# 提取 Publisher 字符串
df_meta["publisher_str"] = df_meta["details_dict"].apply(
    lambda d: d.get("Publisher") if isinstance(d, dict) else None
)

# 提取括号里的 4 位年份
def extract_year(publisher):
    if not isinstance(publisher, str):
        return None
    match = re.search(r'\((?:.*?)(\d{4})\)', publisher)
    if match:
        return int(match.group(1))
    return None

df_meta["publish_year"] = df_meta["publisher_str"].apply(extract_year)

# 丢弃没有年份的记录
df_year = df_meta.dropna(subset=["publish_year"]).copy()

# 按 decade 聚类
df_year["decade"] = (df_year["publish_year"] // 10) * 10

# 统计 top 5 decades
top5_decades = df_year["decade"].value_counts().sort_values(ascending=False).head(5)

# 打印结果
print("📚 出版年代 Top 5：")
print(top5_decades)



📚 出版年代 Top 5：
decade
2010.0    84
2000.0    46
2020.0    21
1990.0    17
1980.0    11
Name: count, dtype: int64


In [16]:
import pandas as pd
import re
import ast

# ==== 2. 从 df_meta 中提取出版年份 ====
def parse_details(details):
    if isinstance(details, dict):
        return details
    try:
        return ast.literal_eval(details)
    except:
        return {}

def extract_year(publisher):
    if not isinstance(publisher, str):
        return None
    match = re.search(r'\((?:.*?)(\d{4})\)', publisher)
    if match:
        return int(match.group(1))
    return None

df_meta["details_dict"] = df_meta["details"].apply(parse_details)
df_meta["publisher_str"] = df_meta["details_dict"].apply(
    lambda d: d.get("Publisher") if isinstance(d, dict) else None
)
df_meta["publish_year"] = df_meta["publisher_str"].apply(extract_year)
df_year = df_meta.dropna(subset=["publish_year"]).copy()
df_year["decade"] = (df_year["publish_year"] // 10) * 10

# ==== 3. 合并 df_review，加上 decade 信息 ====
df_review_with_decade = pd.merge(
    df_review[["parent_asin", "rating"]],
    df_year[["parent_asin", "decade"]],
    on="parent_asin",
    how="inner"
)

# ==== 4. 聚合：每个年代至少 10 本书被评分 ====
grouped = df_review_with_decade.groupby("decade")
book_counts = grouped["parent_asin"].nunique()
avg_ratings = grouped["rating"].mean()

df_filtered = pd.DataFrame({
    "book_count": book_counts,
    "avg_rating": avg_ratings
}).reset_index()

df_filtered = df_filtered[df_filtered["book_count"] >= 10]
df_filtered = df_filtered.sort_values(by="avg_rating", ascending=False)
print(df_filtered)
# ==== 5. 输出结果 ====
if not df_filtered.empty:
    top_decade = df_filtered.iloc[0]
    print(f"\n🏆 平均评分最高的年代是：{int(top_decade['decade'])}s")
    print(f"📚 有评分的图书数量：{top_decade['book_count']}")
    print(f"⭐ 平均评分：{top_decade['avg_rating']:.2f}")
else:
    print("⚠️ 没有任何年代满足至少 10 本书有评分的条件。")


   decade  book_count  avg_rating
8  2020.0          21    4.663636
7  2010.0          84    4.619440
6  2000.0          46    4.273684
4  1980.0          11    4.208333
5  1990.0          17    3.838235

🏆 平均评分最高的年代是：2020s
📚 有评分的图书数量：21.0
⭐ 平均评分：4.66


In [10]:
import pandas as pd
from collections import Counter
import ast

# 读取 JSONL 文件
df = df_meta

# 确保 categories 是列表类型（有些可能是字符串或缺失）
def safe_parse(cat):
    if isinstance(cat, list):
        return cat
    try:
        return ast.literal_eval(cat)
    except:
        return []

df["categories"] = df["categories"].apply(safe_parse)

# 扁平化所有类别
all_categories = [cat for sublist in df["categories"].dropna() for cat in sublist]

# 统计频率
cat_counter = Counter(all_categories)
top10 = cat_counter.most_common(10)

# 输出结果
print("📊 Top 10 most common categories:")
for i, (cat, count) in enumerate(top10, 1):
    print(f"{i}. {cat} — {count} entries")



📊 Top 10 most common categories:
1. Books — 186 entries
2. Literature & Fiction — 40 entries
3. Children's Books — 25 entries
4. History — 15 entries
5. Genre Fiction — 14 entries
6. Mystery, Thriller & Suspense — 13 entries
7. Arts & Photography — 12 entries
8. Science Fiction & Fantasy — 9 entries
9. Thrillers & Suspense — 8 entries
10. Engineering & Transportation — 8 entries


In [21]:
import pandas as pd
import ast

# ==== 2. 安全解析 JSON 格式列 ====
def parse_details(details):
    if isinstance(details, dict):
        return details
    try:
        return ast.literal_eval(details)
    except:
        return {}

def parse_categories(cats):
    if isinstance(cats, list):
        return cats
    try:
        return ast.literal_eval(cats)
    except:
        return []

df_meta["details_dict"] = df_meta["details"].apply(parse_details)
df_meta["categories"] = df_meta["categories"].apply(parse_categories)

# ==== 3. 提取语言信息 ====
df_meta["language"] = df_meta["details_dict"].apply(
    lambda d: d.get("Language") if isinstance(d, dict) else None
)

# ==== 4. 合并 review 和 meta ====
df_merged = pd.merge(
    df_review[["parent_asin", "rating"]],
    df_meta[["parent_asin", "title", "language", "categories"]],
    on="parent_asin",
    how="inner"
)

# ==== 5. 聚合每本书的平均评分 ====
df_grouped = df_merged.groupby(["parent_asin", "title", "language"]).agg({
    "rating": "mean",
    "categories": "first"
}).reset_index()

df_grouped.rename(columns={"rating": "avg_rating"}, inplace=True)

# ==== 6. 筛选条件：language=English 且 avg_rating=5 且类别包含 Literature & Fiction ====
df_filtered = df_grouped[
    (df_grouped["language"] == "English") &
    (df_grouped["avg_rating"] == 5.0) &
    (df_grouped["categories"].apply(lambda cats: "Literature & Fiction" in cats))
]

# ==== 7. 展示结果 ====
if not df_filtered.empty:
    print("📚 满足条件的图书如下（语言 = English，类别包含 Literature & Fiction，平均评分 = 5）：\n")
    display(df_filtered[["title", "language", "categories", "avg_rating"]])
else:
    print("⚠️ 没有找到符合条件的图书。")



📚 满足条件的图书如下（语言 = English，类别包含 Literature & Fiction，平均评分 = 5）：



,title,language,categories,avg_rating
39,The Sludge,English,"[Books, Literature & Fiction, Genre Fiction]",5.0
57,Something That Feels Like Truth (Switchgrass B...,English,"[Books, Literature & Fiction, Short Stories & ...",5.0
83,Kennebago Moments,English,"[Books, Literature & Fiction, Poetry]",5.0
85,Hollywood Confessions: Hollywood Headlines Boo...,English,"[Books, Literature & Fiction, Genre Fiction]",5.0
89,Forged in Blood (Freehold),English,"[Books, Literature & Fiction, Short Stories & ...",5.0
94,Local Honey,English,"[Books, Literature & Fiction]",5.0
101,"Exits, Desires, & Slow Fires",English,"[Books, Literature & Fiction, Poetry]",5.0
107,Fire Cracker,English,"[Books, Literature & Fiction, United States]",5.0
123,Reunion: The Children of Lauderdale Park,English,"[Books, Literature & Fiction, Genre Fiction]",5.0
125,Childe Harold of Dysna,English,"[Books, Literature & Fiction, History & Critic...",5.0


In [8]:
df_review

,rating,title,text,parent_asin,timestamp,helpful_vote,verified_purchase
0,4,Ha! On me! I thought this was a cookbook!,Lucky for me it's more than a cookbook.<br /><...,0385529813,2012-11-25 02:52:05.000,0,False
1,4,Four Stars,Not as developed as Stephanie but I like the c...,1250074274,2015-12-31 21:35:51.000,0,True
2,4,A wonderful adventure in France,I loved this book all the way until the end. I...,1457514575,2013-05-05 17:47:34.000,1,False
3,5,Best beginner book. Been looking for somethin...,Looked online for years for something like thi...,1680450263,2020-08-12 18:06:01.571,0,True
4,4,Referance Guide,Good reference guide for the basics,1889892742,2014-11-14 02:55:08.000,0,True
...,...,...,...,...,...,...,...
1828,5,"love it, love it",The book was all an more than I expected ... l...,1574328522,2017-03-07 22:04:34.000,0,True
1829,5,A good old fashioned ghost story!,I loved this book. It had all the scares and s...,1694621731,2019-12-07 19:54:25.403,0,False
1830,5,Very insightful and helpful,"I found Mr. Ottinger's perspectives, insights,...",0071496769,2012-01-22 20:26:18.000,0,True
1831,5,Powerful & Convicting Reminder of God's Truth ...,Rosaria is an excellent writer with a powerful...,1884527388,2013-04-13 12:35:08.000,0,True


In [27]:
import pandas as pd
import ast
from datetime import datetime


# ==== Ensure timestamp is datetime ====
df_review["timestamp"] = pd.to_datetime(df_review["timestamp"])

# ==== Filter reviews from 2020 onwards ====
df_recent = df_review[df_review["timestamp"] >= pd.Timestamp("2020-01-01")].copy()

# ==== Safe parse of categories ====
def safe_parse(cat):
    if isinstance(cat, list):
        return cat
    try:
        return ast.literal_eval(cat)
    except:
        return []

df_meta["categories"] = df_meta["categories"].apply(safe_parse)

# ==== Filter books with 'History' in category ====
df_meta_history = df_meta[df_meta["categories"].apply(lambda cats: "History" in cats)].copy()

# ==== Merge with filtered review ====
df_merged = pd.merge(
    df_recent[["parent_asin", "rating", "timestamp"]],
    df_meta_history[["parent_asin", "title", "categories"]],
    on="parent_asin",
    how="inner"
)

# ==== Group by book title and categories, and collect review times and ratings ====
df_merged["categories_str"] = df_merged["categories"].apply(lambda x: ", ".join(x) if isinstance(x, list) else str(x))

# ==== Compute average rating per book ====
df_avg = (
    df_merged.groupby(["title", "categories_str"])
    .agg(avg_rating=("rating", "mean"), review_count=("rating", "count"))
    .reset_index()
)

# ==== Filter books with avg rating >= 4.0 ====
df_avg_filtered = df_avg[df_avg["avg_rating"] >= 4.0].copy()

# ==== Join back with review data for inspection ====
df_detailed = pd.merge(
    df_avg_filtered[["title"]],
    df_merged[["title", "timestamp", "rating", "categories"]],
    on="title",
    how="left"
)

# ==== Output ====
print("📚 History books with avg rating ≥ 4.0 since 2020, along with review timestamps and ratings:")
df_detailed



📚 History books with avg rating ≥ 4.0 since 2020, along with review timestamps and ratings:


,title,timestamp,rating,categories
0,Carthage: A Survey of Punic History and Cultur...,2021-02-07 14:44:53.244,5,"[Books, History, Africa]"
1,"Cheer Up, Ben Franklin! (Young Historians)",2023-03-06 13:38:39.404,5,"[Books, Children's Books, History]"
2,Mining Engineers and the American West: The La...,2021-04-13 16:55:28.780,4,"[Books, History, Americas]"
3,Six Gun Sound: The Early History of the Los An...,2022-03-23 14:19:17.671,5,"[Books, History, Americas]"
4,Six Gun Sound: The Early History of the Los An...,2022-12-26 21:59:27.358,5,"[Books, History, Americas]"


In [12]:
latest_time = df_review['timestamp'].max()
print("📅 最迟评论时间是:", latest_time)



📅 最迟评论时间是: 2023-06-14 16:29:39.173000
